In [1]:
import shutil

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3

from utils import *

In [2]:
NUM_POINTS = 4096

coordinate_system = create_coordinate_system()

In [3]:
class Shape():
    
    clouds = []
        
    def __init__(self, o=(0, 0, 0)):
        self.o = o  # Origin
        self.points = o3.Vector3dVector(np.asarray([], dtype=np.float32))
        self.colors = o3.Vector3dVector(np.asarray([], dtype=np.float32))
        self.clouds = []
    
    def draw(self):
        try:
            o3.draw_geometries(list(flatten(self.clouds)))
        except TypeError:
            o3.draw_geometries([self.clouds])
            
    def combine_clouds(self, clouds):
        points = []
        colors = []
        for cloud in list(flatten(clouds)):
            points.append(cloud.points)
            colors.append(cloud.colors)
        points = np.concatenate(points)
        colors = np.concatenate(colors)
        cloud.points = o3.Vector3dVector(np.asarray(points, dtype=np.float32))
        cloud.colors = o3.Vector3dVector(np.asarray(colors, dtype=np.float32))
        return cloud
    
    def rotate(self, axis, theta):
        for cloud in self.clouds:
            points = np.copy(np.asarray(cloud.points))
            points = [np.dot(rotation_matrix(axis, theta), x) for x in points]
            cloud.points = o3.Vector3dVector(np.asarray(points, dtype=np.float32))
            
    def random_rotate(self):
        q, r = np.linalg.qr(np.random.randn(3, 3))
        for cloud in self.clouds:
            points = np.asarray(cloud.points)
            points = [q @ x for x in points]
            cloud.points = o3.Vector3dVector(np.asarray(points, dtype=np.float32))

In [4]:
class Sphere(Shape):
    
    def __init__(self, o=(0, 0, 0), r=1):
        super().__init__(o)
        self.r = r
        self.create_points()
        
    def create_points(self):
        u = np.random.uniform(0, 1, NUM_POINTS)
        v = np.random.uniform(0, 1, NUM_POINTS)
        thetas = 2 * np.pi * u
        phis = np.arccos(2 * v - 1)

        x = self.o[0] + self.r * np.cos(thetas) * np.sin(phis)
        y = self.o[1] + self.r * np.sin(thetas) * np.sin(phis)
        z = self.o[2] + self.r * np.cos(phis)
        points = list(zip(x, y, z))
        cloud = make_cloud(points, color=RED)
        self.clouds.append(cloud)

In [5]:
class Circle(Shape):
    
    def __init__(self, o=(0, 0, 0), r=1):
        super().__init__(o)
        self.r = r
        self.create_points()

    def create_points(self):
        thetas = np.random.uniform(0, 2 * np.pi, NUM_POINTS)
        phis = np.random.uniform(0, 2 * np.pi, NUM_POINTS)
        l = self.r * np.sqrt(np.random.uniform(0, 1, NUM_POINTS))
        x = self.o[0] + l * np.cos(thetas)
        y = self.o[1] + np.array([0 for _ in range(NUM_POINTS)])
        z = self.o[2] + l * np.sin(thetas)
        points = list(zip(x, y, z))
        cloud = make_cloud(points, color=BLUE)
        self.clouds.append(cloud)

In [6]:
class Cylinder(Shape):
    
    def __init__(self, o=(0, 0, 0), r=1, l=2):
        super().__init__(o)
        self.r = r
        self.l = l
        self.create_points()
        
    def create_points(self):
        thetas = np.random.uniform(0, 2 * np.pi, NUM_POINTS)
        x = self.o[0] + self.r * np.cos(thetas)
        y = self.o[1] + np.random.uniform(0, self.l, NUM_POINTS)
        z = self.o[2] + self.r * np.sin(thetas)
        points = list(zip(x, y, z))
        cylinder = make_cloud(points, color=BLUE)
        bot_cap = Circle(o=self.o)
        top_cap_origin = (self.o[0], self.o[1] + self.l, self.o[2])
        top_cap = Circle(o=top_cap_origin)
        cloud = self.combine_clouds([cylinder, bot_cap.clouds, top_cap.clouds])
        self.clouds.append(cloud)
        return cloud

In [7]:
def is_in_sphere(point):
    """Assume a sphere radius of 3 and origin at (0, 0, 0). To update write a wrapper function."""
    x, y, z = point
    return x ** 2 + y ** 2 + z ** 2 > 3 ** 2

In [8]:
class Spike(Shape):
    
    def __init__(self, o=(0, 0, 0)):
        super().__init__(o)
        sphere_radius = 3
        sphere = Sphere(r=sphere_radius)
        cylinder = Cylinder(o=(0, sphere_radius - 0.2, 0), l=3)
        cylinder_cloud = cylinder.clouds[0]
        points = np.asarray(cylinder_cloud.points)
        points = list(filter(is_in_sphere, points))
        cylinder_cloud.points = o3.Vector3dVector(np.asarray(points, dtype=np.float32))
        set_colors(cylinder_cloud, BLUE)
        
        cylinder.random_rotate()
        self.clouds.append([sphere.clouds, cylinder.clouds])

In [9]:
#spike = Spike()
#spike.draw()

In [10]:
def write(clouds, name, annotation):
    for idx, cloud in enumerate(clouds):
        o3.write_point_cloud(f'{annotation.as_posix()}/{name}_{idx}.xyzrgb', cloud)
        
    for file in annotation.iterdir():
        filename = annotation / (file.stem + '.txt')
        if not filename.is_file():
            shutil.copyfile(file, filename)

    for file in [f for f in annotation.iterdir() if f.suffix == '.xyzrgb']:
        os.remove(file.as_posix())

def write_out(directory):
    spike = Spike()
    spheres = spike.clouds[0][0]
    cylinders = spike.clouds[0][1]

    annotation = directory / 'annotation'
    annotation.mkdir(exist_ok=True, parents=True)
    write(spheres, 'sphere', annotation)
    write(cylinders, 'cylinder', annotation)

    filename = directory / (directory.stem + '.txt')
    with open(filename.as_posix(), 'w') as outfile:
        for file in Path(annotation).iterdir():
            with open(file, 'r') as infile:
                outfile.write(infile.read())

In [12]:
for test_or_train in ['test', 'train']:
    for idx in range(10):
        directory = Path(data_dir) / 'scenes' / test_or_train / 'uniform' / f'spike_{idx}'
        print(f'writing {directory.as_posix()}')
        write_out(directory)

writing data/scenes/test/uniform/spike_0
writing data/scenes/test/uniform/spike_1
writing data/scenes/test/uniform/spike_2
writing data/scenes/test/uniform/spike_3
writing data/scenes/test/uniform/spike_4
writing data/scenes/test/uniform/spike_5
writing data/scenes/test/uniform/spike_6
writing data/scenes/test/uniform/spike_7
writing data/scenes/test/uniform/spike_8
writing data/scenes/test/uniform/spike_9
writing data/scenes/train/uniform/spike_0
writing data/scenes/train/uniform/spike_1
writing data/scenes/train/uniform/spike_2
writing data/scenes/train/uniform/spike_3
writing data/scenes/train/uniform/spike_4
writing data/scenes/train/uniform/spike_5
writing data/scenes/train/uniform/spike_6
writing data/scenes/train/uniform/spike_7
writing data/scenes/train/uniform/spike_8
writing data/scenes/train/uniform/spike_9
